<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Деревья поиска](#toc1_)    
  - [Дерево поиска](#toc1_1_)    
  - [АВЛ - дерево](#toc1_2_)    
  - [Операции на АВЛ-дереве.](#toc1_3_)    
    - [Обращение по индексу](#toc1_3_1_)    
    - [Склеивание](#toc1_3_2_)    
    - [Разрезание на части](#toc1_3_3_)    
  - [Применения](#toc1_4_)    
- [Сплей-деревья](#toc2_)    
  - [Детали реализации](#toc2_1_)    
  - [Анализ](#toc2_2_)    
- [Задача. Обход двоичного дерева](#toc3_)    
  - [Задача. Проверка свойства дерева поиска](#toc3_1_)    
  - [Задача. Проверка более общего свойства дерева поиска](#toc3_2_)    
  - [Задача. Множество с запросами суммы на отрезке](#toc3_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Деревья поиска](#toc0_)

Когда х-таблицы недостаточно: когда не только сравнение (есть/нет), но важен и порядок (следующий эл-т, порядковая статистика, эл-ты больше заданного и т.д.).  
Упорядоченный массив может быть недостаточно быстр из-за операции вставки, которая в хужшем случае O(n) из-за необходимости сдвигать элементы.  

## <a id='toc1_1_'></a>[Дерево поиска](#toc0_)

Для каждого поддерева выполняется условие, что правый потомок больше (нестрого) корня, левый - меньше (строго) корня.  
Время поиска мин/макс, следующего, удаления, вставки O(высота).  В общем случае высота будет зависеть от порядка вставки элементов (1, 2, 3, 4, 5 - все пойдут в правое поддерево, и высота будет n)        

## <a id='toc1_2_'></a>[АВЛ - дерево](#toc0_)

АВЛ-дерево (1962г Адельсон-Вельский, Ландис).  
Балансируем дерево так, чтобы в каждой вершине высота правого и левого поддеревьев отличались не более чем на 1.  

    Лемма: высота АВЛ-дерева с n вершин не больше O(logn).  
    Док-во: O(n) = h(n) nmin = 2^(h-1) nmax = 2^h - 1 -> h(n)= [log2(n-1)..log2(n)+1] ч.и т.д.

Балансируется "вращениями":  

    A(a), B(a), C(b) - поддеревья ABC с корнями в ab: b(C,a(AB)) -> a(A,b(BC)) : A < a < B < b < C

Высоты поддеревье хранятся в каждой вершине, после врашения высоты пересчитываются.   
Вставка/удаление могут изметить высоту поддерева на 1 и баланс может измениться максимум на 2.  
Исправлять это вращениями идя снизу вверх. 
Каждое врашение O(1) всего шагов O(logn).


## <a id='toc1_3_'></a>[Операции на АВЛ-дереве.](#toc0_)

### <a id='toc1_3_1_'></a>[Обращение по индексу](#toc0_)

В каждой вершине хранится количество элементов в его поддереве  

    OrderStatistics(v, k):
    leftsize = v.left.size
    if k == leftsize + 1:
        return v
    if k < leftsize + 1:
        return OrderStatistics(v.left, k)
    else:
        return OrderStatistics(v.right, k - leftsize - 1)

Для этого при изменении детей вершины нужно обновлять size

    UpdateSize(v):
    v.size = v.left.size + v.right.size
    

### <a id='toc1_3_2_'></a>[Склеивание](#toc0_)

Вход: АВЛ-деревья Т1, Т2: T1 < T2 для любой вершины  
Выход: новое АВЛ-дерево T  

    MergeWithRoot(v1, v2, T):           # через промежуточную вершину (баланса пока нет)
    T.left = v1
    T.right = v2
    v1.parent = T
    v2.parent = T 
    return T

    Merge(v1, v2):                      # выделение промежуточной вершины (баланса пока нет)
    T = Max(v1)
    Delete(T)
    MergeWithRoot(v1, v2, T)
    return T

    AVLMergeWithRoot(v1, v2, T):
    if |v1.height - v2.height| <= 1:
        MergeWithRoot(v1. v2. T)
        update T.height                 # макс(v1,v2) +1
        return T
    else if v1.height > v2.height:      # по правому (большему) сыну v1 рекурсивно, пока разница высот c v2 не будет <= 1    
        Tx = AVLMergeWithRoot(v1.right. v2, T)
        v1.right = Tx
        Tx.parent = v1
        return Rebalance(v1)
    else ...                            # аналогично наоборот

Время работы O(|v1.height - v2.height| + 1)  

### <a id='toc1_3_3_'></a>[Разрезание на части](#toc0_)

Вход: АВЛ-дерево Т, ключ k
Выход: АВЛ-деревья Т1, Т2: T1 <= k < T2  

    Split(v, k):
    if v = null:
        return (null, null)
    if k < v.key:
        (v1, v2) = Split(v.left, k)
        v2x = MergeWithRoot(v2, v.right, v)
        return (v1, v2x)
    else: 
    ...

Время работы sum O(|hi - hi-1| + 1) = O(hmax) = O(logn)

## <a id='toc1_4_'></a>[Применения](#toc0_)

1. Структуры данных с поддержкой: вставки, поиска, удаления ключа + поиска суммы ключей на отрезке \[l,r]  

В вершине поддерживается поле sum. Пересчитывается при изменениях в вершине (ключ вершины + суммы потомков).  
Для поиска на отрезке - отрезать l-1 слева, отрезать r справа, сумма окажется в корне.  
После получения ответа, нужно обратно все склеить.  


2. Массив с поддержкой «вырезать и переставить» 

Ключей нет, нам нужно искать элемент на k-м месте.
Трюк заключается в "неявных ключах"...  
Допустим массив такого вида, в котором надо найти элемент (не по ключу, а по индексу):

    12345678    # индекс - число элементов поддерева (размер)
    bacbdabc    # элементы

... на самом деле этого массива нет, а есть дерево, где в каждой вершине (т.е. элементе) поддерживается актуальное значение количества элементов в соотв. поддереве. Тогда поиск элемента с индексом k примерно такой. 
    
    Поиск(k, корень):
    Если количество элементов в корне равно k:
        вернуть корень
    Иначе если количество элементов в левом пд меньше k:
        то идем в правое поддерево корня и ищем там рекурсивно (k - количество в левом пд - 1 (корень))-й элемент, т.е. выполняем Поиск(k - корень.сумм.лев - 1, корень правого пд )
    Иначе:
        Поиск(k, корень левого пд).

И тогда разрезание и перевставка:  

    Reorder(v, i):
    (v1, v2) = Split(v, i)
    Merge(v2, v1)

    


# <a id='toc2_'></a>[Сплей-деревья](#toc0_)

Или косое дерево (1985г Слитор - Тарьян)
Поддерживает сбалансированность "в среднем" (АВЛ - сбалансровано всегда), может разбалансироваться, т.е. гарантирует logn, но уже не в худшем случае, а в среднем.  
Основная операция - Splay(u): вращениями поднимает поддерево наверх.  

    Лемма: существует такая функция потенциала, определенная на бинарных деревьях и принимающая значения [0, O(nlogn)], что учетная стоимость 
    (сумма истинного времени работы и изменения в ее результате потенциала) операций Slpay относительно нее есть O(logn)  
    Док-во: Повторим m>n раз выбор самой глубокой вершины, вызок для нее Splay (вывод в корень самой дальней вершины) 
    Суммарное время работы O(mlogn + nlogn) = O(mlogn) - это по предыдущей лемме про АВЛ. Т.е. средняя глубина останется O(mlogn) / m == O(logn).  
    
Те вершины, которые недавно искались, вытаскиваются наверх. Т.е. добавление в дерево подряд ключей по порядку сделает просто список, но первое обращение к самому дальнему ключу вытащит его наверх и при этом подтянет другие в поддеревья в рамках вращений.

## <a id='toc2_1_'></a>[Детали реализации](#toc0_)

Splay(u): пусть u левый сын своего отца, тогда   
-> если отец корень - шаг zig (меняется с отцом, правое поддерево уходит к бывшему отцу налево), 
-> если отец не корень, то шаг zigzig (меняется с дедом, правое дерево уходит бывшему отцу влево, чье правое дерево уходит деду направо) если отец - левый сын деда,  
-> иначе - zigzag.  

    Search(k): спуск по дереву, пусть u - последняя вершина (либо она ключ, либо больше некуда идти), тогда вызвать Splay(u)  
        ... неформально, поднятие последней вершины - оплата работы по ее поиску, типа не зря ходили...
    Insert(k): вставить новую вершину в корен, вызвать для нее Splay()  
    Split(k): u - последняя вершина при поиске k, вызвать Splay(u)
    Merge(v1,v2): u - верш.с макс.ключом в v1; Splay(u); сделаем v2 правым сыном u  
    Remove(u): Splay(u); Merge(u.left, u.right)    

## <a id='toc2_2_'></a>[Анализ](#toc0_)

Потенциал: 

    вес вершины w(u) = # вершин в поддереве с корнем u
    потенциал вершины Ф(u) = round(log2 w(u)) - т.е. не важно 5 или 6 вершин в поддереве, их потенциал одинаков
    потенциал дерева Ф(Т) = sum Ф(u)

-> ( 0 <= Ф(Т) <= nlog2(n) )   

Splay(u) вытягивает u в корень r, тогда учетная степень Splay(u) не более 3(Ф(r) - Ф(u)) + 1  
-> для двойных шагов zigzig zigzag учетная стоимость не более 3(Ф1(u) - Ф(u)), где Ф1 - после поворота  
-> для одинакрных шагов zig - не более 3(Ф1(u) - Ф(u)) + 1  

Строгое док-во запутанное, но время работы всех операций O(logn)

# <a id='toc3_'></a>[Задача. Обход двоичного дерева](#toc0_)

Построить in-order, pre-order и post-order обходы данного двоичного дерева.  
Вход. Двоичное дерево.  
Выход. Все его вершины в трёх разных порядках: in-order, pre-order и post-order.  
Формат входа. Первая строка содержит число вершин n. Вершины дерева пронумерованы числами от 0 до n−1. Вершина 0 является корнем. Каждая из следующих n строк содержит информацию о вершинах 0, 1, . . . , n − 1: i-я строка задаёт числа key i , left i и right i , где key i — ключ вершины i, left i — индекс левого сына вершины i, а right i — индекс правого сына вершины i. Если у вершины i нет одного или обоих сыновей, соответствующее значение равно −1.  
Формат выхода. Три строки: in-order, pre-order и post-order обходы.  
Ограничения. 1 ≤ n ≤ 10^5 ; 0 ≤ key i ≤ 10^9 ; −1 ≤ left i , right i ≤ n − 1. Гарантируется, что вход задаёт корректное двоичное дерево: в частности, если left i != −1 и right i != −1, то left i != right i ; никакая вершина не является сыном двух вершин; каждая вершина является потомком корня.  

In-order обход соответствует следующей рекурсивной процедуре, получающей на вход корень v текущего поддерева: произвести рекурсивный вызов для v.left, напечатать v.key, произвести рекурсивный вызов для v.right. Pre-order обход: напечатать v.key, произвести рекурсивный вызов для v.left, произвести рекурсивный вызов для v.right. Post-order: произвести рекурсивный вызов для v.left, произвести рекурсивный вызов для v.right, напечатать v.key.

In [48]:
SAMPLES = "5\n4 1 2\n2 3 4\n5 -1 -1\n1 -1 -1\n3 -1 -1", "10\n0 7 2\n10 -1 -1\n20 -1 6\n30 8 9\n40 3 -1\n50 -1 -1\n60 1 -1\n70 5 4\n80 -1 -1\n90 -1 -1", 
OUTPUTS = ... 
READER = (x for x in SAMPLES[1].split('\n')); input = lambda: next(READER)

from collections import namedtuple

Node = namedtuple("Node", ["key", "left", "right"])

class Tree():
    def __init__(self):
        self.tree = []
        self.preorder = []
        self.inorder = []
        self.postorder = []

    def new(self, key, left, right):
        self.tree.append(Node(key, left, right))
    
    def walk(self, node=0):
        if node != -1:
            self.preorder.append(self.tree[node].key)
            self.walk(self.tree[node].left)
            self.inorder.append(self.tree[node].key)
            self.walk(self.tree[node].right)
            self.postorder.append(self.tree[node].key)

def main():
    n = int(input())
    tree = Tree()
    for _ in range(n):
        tree.new(*list(map(int, input().split())))

    tree.walk()
    print(*tree.inorder)
    print(*tree.preorder)
    print(*tree.postorder)

main()

50 70 80 30 90 40 0 20 10 60
0 70 50 40 30 80 90 20 60 10
50 80 90 30 40 70 10 60 20 0


In [2]:
SAMPLES = "5\n4 1 2\n2 3 4\n5 -1 -1\n1 -1 -1\n3 -1 -1", "10\n0 7 2\n10 -1 -1\n20 -1 6\n30 8 9\n40 3 -1\n50 -1 -1\n60 1 -1\n70 5 4\n80 -1 -1\n90 -1 -1", 
OUTPUTS = ...
READER = (x for x in SAMPLES[1].split('\n')); input = lambda: next(READER)

from collections import namedtuple

Node = namedtuple("Node", ["key", "left", "right"])
          
def inorder(current=0):
    stack = []
    while stack or current != -1:
        if stack:
            current = stack.pop()
            yield tree[current].key         
            current = tree[current].right   # перейти в правое п.д.
        while current != -1:
            stack.append(current)
            current = tree[current].left    # идти налево до конца

def preorder(current=0):
    stack = []
    while stack or current != -1:
        if stack:
            current = stack.pop()           # перейти в правое п.д.
        while current != -1:
            yield tree[current].key
            if tree[current].right != -1:
                stack.append(tree[current].right)   # заполнить стек правыми
            current = tree[current].left    # к обработке - левые

def postorder(current=0):
    stack = []
    while stack or current != -1:
        if stack:
            current = stack.pop()
            if stack and tree[current].right == stack[-1]: # если "петля" ...
                current = stack.pop()       # ... то перейти на правое п.д.
            else:
                yield tree[current].key
                current = -1                # если стек пустой, то конец
        while current != -1:
            stack.append(current)
            if tree[current].right != -1:   # если есть правое п.д. то в стеке будет ...
                stack.append(tree[current].right)
                stack.append(current)       # "петля", за которую зацепится при обработке
            current = tree[current].left    # идти налево до конца

tree = []

n = int(input())
for _ in range(n):
    tree.append(Node(*list(map(int, input().split()))))

print(*inorder())
print(*preorder())
print(*postorder())



50 70 80 30 90 40 0 20 10 60
0 70 50 40 30 80 90 20 60 10
50 80 90 30 40 70 10 60 20 0


## <a id='toc3_1_'></a>[Задача. Проверка свойства дерева поиска](#toc0_)

Проверить, является ли данное двоичное дерево деревом поиска.
Вход. Двоичное дерево.  
Выход. Проверить, является ли оно корректным деревом поиска: верно ли, что для любой вершины дерева её ключ больше всех ключей в левом поддереве данной вершины и меньше всех ключей в правом поддереве.  
Формат входа. Первая строка содержит число вершин n. Вершины дерева пронумерованы числами от 0 до n−1. Вершина 0 является корнем. Каждая из следующих n строк содержит информацию о
вершинах 0, 1, . . . , n − 1: i-я строка задаёт числа key i , left i и right i , где key i — ключ вершины i, left i — индекс левого сына вершины i, а right i — индекс правого сына вершины i. Если у вершины i нет одного или обоих сыновей, соответствующее значение равно −1.  
Формат выхода. Выведите «CORRECT», если дерево является корректным деревом поиска, и «INCORRECT» в противном случае.  
Ограничения. 0 ≤ n ≤ 10 5 ; −2 31 < key i < 2 31 − 1; −1 ≤ left i , right i ≤ n − 1. Гарантируется, что вход задаёт корректное двоичное дерево: в частности, если left i != −1 и right i != −1, то left i != right i; никакая вершина не является сыном двух вершин; каждая вершина является потомком корня. 




In [3]:
SAMPLES = "3\n2 1 2\n1 -1 -1\n3 -1 -1", "3\n1 1 2\n2 -1 -1\n3 -1 -1", "0", "5\n1 -1 2\n2 -1 2\n3 -1 3\n4 -1 4\n5 -1 -1"
OUTPUTS = "CORRECT", "INCORRECT", "CORRECT", "CORRECT" # не обязательно сбалансированное
READER = (x for x in SAMPLES[2].split('\n')); input = lambda: next(READER)

from collections import namedtuple

Node = namedtuple("Node", ["key", "left", "right"])

def inorder(current=0):
    if tree:
        stack = []
        while stack or current != -1:
            if stack:
                current = stack.pop()
                yield tree[current].key         
                current = tree[current].right   # перейти в правое п.д.
            while current != -1:
                stack.append(current)
                current = tree[current].left    # идти налево до конца


tree = []
correct = True

n = int(input())
for _ in range(n):
    tree.append(Node(*list(map(int, input().split()))))

nodes = inorder()
try:
    last = next(nodes)
except StopIteration:
    last = -1

for node in nodes:
    if node < last:
        correct = False
    last = node

print("CORRECT" if correct else "INCORRECT")


CORRECT


## <a id='toc3_2_'></a>[Задача. Проверка более общего свойства дерева поиска](#toc0_)

Данная задача полностью аналогична предыдущей, но проверять теперь нужно более общее свойство. Дереву разрешается содержать равные ключи, но они всегда должны находиться в правом поддереве.  
Формально, двоичное дерево называется деревом поиска, если для любой вершины её ключ больше всех ключей из её левого поддерева и не меньше всех ключей из правого поддерева.

In [4]:
SAMPLES = ["3\n2 1 2\n1 -1 -1\n3 -1 -1",        #0 1  
            "3\n1 1 2\n2 -1 -1\n3 -1 -1",       #1 0
            "3\n2 1 2\n1 -1 -1\n2 -1 -1",       #2 1
            "3\n2 1 2\n2 -1 -1\n3 -1 -1",       #3 0
            "1\n2147483647 -1 -1",              #4 1
            "10\n0 7 2\n10 -1 -1\n20 -1 6\n30 8 9\n40 3 -1\n50 -1 -1\n60 1 -1\n70 5 4\n80 -1 -1\n90 -1 -1",     #5 0
            "0",                                #6 1
            "2\n-2147483648 1 -1\n-1000000000 -1 -1",   #7 0
            "3\n2 -1 1\n3 2 -1\n3 -1 -1"]               #8 0
OUTPUTS = "CORRECT", "INCORRECT", "CORRECT", "INCORRECT", "CORRECT", "CORRECT"
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)

class Node():
    def __init__(self, *args):
        self.max = -float('inf')
        self.min = float('inf')
        self.correct = True
        self.key, self.left, self.right = args

def minmax(node):
    l = node if tree[node].left == -1 else tree[node].left
    r = node if tree[node].right == -1 else tree[node].right
    if l == r: 
        tree[node].max = tree[node].key
        tree[node].min = tree[node].key
    else:
        tree[node].correct = tree[l].max < tree[node].key and tree[node].key <= tree[r].min
        tree[node].max = max(tree[node].key, tree[l].max, tree[r].max)
        tree[node].min = min(tree[node].key, tree[l].min, tree[r].min)

def postorder(current=0):
    if tree:
        stack = []
        while stack or current != -1:
            if stack:
                current = stack.pop()
                if stack and tree[current].right == stack[-1]: # если "петля" ...
                    current = stack.pop()       # ... то перейти на правое п.д.
                else:
                    minmax(current)
                    current = -1                # если стек пустой, то конец
            while current != -1:
                stack.append(current)
                if tree[current].right != -1:   # если есть правое п.д. то в стеке будет ...
                    stack.append(tree[current].right)
                    stack.append(current)       # "петля", за которую зацепится при обработке
                current = tree[current].left    # идти налево до конца


tree = []

n = int(input())
for _ in range(n):
    tree.append(Node(*list(map(int, input().split()))))

postorder()
print("INCORRECT" if False in [node.correct for node in tree] else "CORRECT")

print(*[node.correct for node in tree])
print(*[(node.max, node.min) for node in tree])

CORRECT
True True True
(3, 1) (1, 1) (3, 3)


## <a id='toc3_3_'></a>[Задача. Множество с запросами суммы на отрезке](#toc0_)

Реализуйте структуру данных для хранения множества целых чисел, поддерживающую запросы добавления, удаления, поиска, а также суммы на отрезке. На вход в данной задаче будет дана последовательность таких запросов. Чтобы гарантировать, что ваша программа обрабатывает каждый запрос по мере поступления (то есть онлайн), каждый запрос будет зависеть от результата выполнения одного из предыдущих запросов. Если бы такой зависимости не было, задачу можно было бы решить оффлайн: сначала прочитать весь вход и сохранить все запросы в каком-нибудь виде, а потом прочитать вход ещё раз, параллельно отвечая на запросы.  

Формат входа.   
Изначально множество пусто. Первая строка содержит число запросов n. Каждая из n следующих строк содержит запрос в одном из следующих четырёх форматов:  
• + i: добавить число f (i) в множество (если оно уже есть, проигнорировать запрос);  
• - i: удалить число f (i) из множества (если его нет, проигнорировать запрос);  
• ? i: проверить принадлежность числа f (i) множеству;  
• s l r: посчитать сумму всех элементов множества, попадающих в отрезок \[f (l), f (r)].  
Функция f определяется следующим образом. Пусть s — результат последнего запроса суммы на отрезке (если таких запросов ещё не было, то s = 0). Тогда  
    
    f (x) = (x + s) mod 1 000 000 001

Формат выхода.  
Для каждого запроса типа ? i выведите «Found» или «Not found». Для каждого запроса суммы выведите сумму всех элементов множества, попадающих в отрезок \[f (l), f (r)]. Гарантируется, что во всех тестах f (l) ≤ f (r).

Ограничения.  
1 ≤ n ≤ 10^5 ; 0 ≤ i ≤ 10^9.

In [5]:
SAMPLES = ["15\n? 1\n+ 1\n? 1\n+ 2\ns 1 2\n+ 1000000000\n? 1000000000\n- 1000000000\n? 1000000000\ns 999999999 1000000000\n- 2\n? 2\n- 0\n+ 9\ns 0 9", 
           "5\n? 0\n+ 0\n? 0\n- 0\n? 0",
           "5\n+ 491572259\n? 491572259\n? 899375874\ns 310971296 877523306\n+ 352411209",
           "6\n+ 10\n+ 5\n+ 14\n+ 3\n+ 11\n+ 12",
           "6\n+ 5\n+ 15\n+ 10\n+ 8\n+ 7\ns 4 9"]           
OUTPUTS = ...
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)
# штош, пишем сначала бинарное дерево поиска без оптимизаций и всего такого, шоб хоть как-то

s = 0
MOD = 10 ** 9 + 1
bormuliator = lambda x: (x + s) % MOD
# free_idx = []

class Node():
    def __init__(self, key, parent=-1):
        assert 0 <= key < MOD
        self.parent = parent
        self.root = 0
        self.left = -1
        self.right = -1
        self.key = key
    
    def __repr__(self):
        return "{}:{}({},{})".format(self.key, self.parent, self.left, self.right)

class Tree(list):
    def __bool__(self):
        if not any(self) and len(self) > 0:
            self.__init__()                     # схлопываем накопившиесы None
        return True if any(self) else False

def find(key, node=0):
    if tree:
        while node != -1:
            if key < tree[node].key:
                node = tree[node].left
            elif key > tree[node].key:
                node = tree[node].right
            else:
                print("Found")
                return Node
    print("Not found")

def add(key, node=0):
    if not tree:
        # if free_idx:
        #     tree[free_idx.pop()] = Node(key, parent=node)
        # else: 
        tree.append(Node(key)) 
    else:
        if key < tree[node].key:
            if tree[node].left != -1: 
                add(key, tree[node].left)
            else:
                # idx = len(tree)
                # if free_idx: 
                #     idx = free_idx.pop()
                #     tree[idx] = Node(key, parent=node)
                # else: 
                tree.append(Node(key, parent=node))
                tree[node].left = len(tree) - 1
        if key > tree[node].key:
            if tree[node].right != -1: 
                add(key, tree[node].right)
            else: 
                # idx = len(tree)
                # if free_idx: 
                #     idx = free_idx.pop()
                #     tree[idx] = Node(key, parent=node)
                # else: 
                tree.append(Node(key, parent=node))
                tree[node].right = len(tree) - 1
        else:
            return

def inorder(node=0, func=lambda x: tree[x].key):
    if tree:
        stack = []
        while stack or node != -1:
            if stack:
                node = stack.pop()
                yield func(node)
                node = tree[node].right   # перейти в правое п.д.
            while node != -1:
                stack.append(node)
                node = tree[node].left    # идти налево до конца

def summ(l, r):
    global s
    assert l<=r
    s = sum([x for x in inorder() if l <= x <= r])
    print(s)    # на пустом ОК

def remove(key, node=0):
    if tree:
        if key < tree[node].key and tree[node].left != -1:
            remove(key, tree[node].left)
        if key > tree[node].key and tree[node].right != -1:
            remove(key, tree[node].right)
        if key == tree[node].key:
            if tree[node].left == -1 and tree[node].right == -1:   # нет потомков: удаляем узел, родителю зануляем ссылку
                if tree[node].parent != -1:
                    if tree[tree[node].parent].left == node:
                        tree[tree[node].parent].left = -1
                    elif tree[tree[node].parent].right == node:
                        tree[tree[node].parent].right = -1
                    else: raise Error
                tree[node] = None
                # free_idx.append(node)
            elif tree[node].left * tree[node].right < 0:    # один потомок: подтягиваем поддерево потомка
                c = tree[node].right if (tree[node].left == -1) else tree[node].left
                tree[node].key = tree[c].key 
                tree[node].left = tree[c].left 
                tree[node].right = tree[c].right
                if tree[c].left != -1: tree[tree[c].left].parent = tree[c].parent      # обновляем родителей
                if tree[c].right != -1: tree[tree[c].right].parent = tree[c].parent
                tree[c] = None
                # free_idx.append(c)
            else:                                           # два потомка: самый левый в пр.п.д. меняем ключи с удаляемым, если осталось пр.п.д. - подтянуть вверх
                r_min = next(inorder(node=tree[node].right, func=lambda x: x))
                tree[node].key = tree[r_min].key

                if tree[r_min].right != -1:                                     # если было пр.п.д. - подтянуть к родителю
                    if tree[r_min].parent == node:
                        tree[tree[r_min].parent].right = tree[r_min].right      # у rmin если есть то только пр. потомок
                    else:
                        tree[tree[r_min].parent].left = tree[r_min].right
                    tree[tree[r_min].right].parent = tree[r_min].parent

                if tree[r_min].left == -1 and tree[r_min].right == -1:         # ... если не было
                    if tree[r_min].parent == node:                             # rmin с пр. родителем только у удаляемого
                        tree[tree[r_min].parent].right = -1                     
                    else:
                        tree[tree[r_min].parent].left = -1
                tree[r_min] = None
                # free_idx.append(r_min)

def run():
    commands = {'+': add, '?': find, '-': remove, 's': summ}
    n = int(input())
    for _ in range(n):
        cmd, *args = input().split()
        args = list(map(bormuliator, map(int, args)))
        # print(cmd, *args, tree)
        commands[cmd](*args)

tree = Tree()
run()

Not found
Found
3
Found
Not found
1
Not found
10


In [6]:
tree = Tree()
import random
n = 10 ** 5

def test():
    I = [random.randint(1,n) for x in range(n)] * 2
    for x in I:
        random.choice([add, remove])(x)
    for x in I:
        add(x)
    for x in reversed(I):
        remove(x)

# первый рабочий вариант
%timeit test()

9.18 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def traverse(t):
    thislevel = [t[0]]
    while thislevel:
        nextlevel = list()
        for n in thislevel:
            print(n.key, end=' ')
            if n.left != -1: nextlevel.append(t[n.left])
            if n.right != -1: nextlevel.append(t[n.right])
        print()
        thislevel = nextlevel

# traverse(tree)

In [47]:
# ООП дерево
class NodeOO:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
    
    def __repr__(self):
        if self.left.node == self.right.node == None:
            return str(self.key)
        else:
            return "{}({},{})".format(self.key, self.left.node or '', self.right.node or '')

class BSTreeOO():
    def __init__(self):
        self.node = None
        self.height = -1

    def __repr__(self):
        return repr(self.node)

    def height(self):
        return self.node.height if self.node else 0
        
    def infix_walk(self, node, func=lambda node: node.key):
        stack = list()
        while stack or node != None:
            if stack:
                node = stack.pop()
                yield func(node)
                node = node.right.node
            while node != None:
                stack.append(node)
                node = node.left.node
    
    def find(self, key):
        node = self.node
        while node != None:
            if key < self[node].key:
                node = node.left
            elif key > node.key:
                node = node.right
            else:
                return node

    def add(self, key):
        node = self.node
        if node == None:
            self.node = NodeOO(key) 
            self.node.left = BSTreeOO() 
            self.node.right = BSTreeOO()        
        elif key < node.key: 
            self.node.left.add(key)
        elif key > node.key: 
            self.node.right.add(key)            

    def summ(self, l, r):
        self.accumulator = sum([x for x in self.infix_walk() if l <= x <= r])
        return self.accumulator 

    def remove(self, key):
        if self.node != None: 
            if self.node.key == key: 
                if self.node.left.node == self.node.right.node == None:     # потомков нет
                    self.node = None
                elif self.node.left.node == None:                           # один потомок: подтянуть справа
                    self.node = self.node.right.node
                elif self.node.right.node == None:                          # один потомок: подтянуть слева
                    self.node = self.node.left.node
                else:                                                       # два потомка
                    right_min = self.right_minimum(self.node)
                    self.node.key = right_min.key 
                    self.node.right.remove(right_min.key)                   # теперь у того уже будет 1 потомок
                return  
            elif key < self.node.key: 
                self.node.left.remove(key)  
            elif key > self.node.key: 
                self.node.right.remove(key)

    def right_minimum(self, node):
        node = node.right.node  
        while node.left != None:
            if node.left.node == None: 
                return node 
            else: 
                node = node.left.node  
        return node 


import random
T = BSTreeOO()
T.add(3), T.add(1), T.add(2), T.add(5), T.add(4)
print(T)

        

3(1(,2),5(4,))
